# 初识神经网络


In [2]:
from keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

mnist 是识别手写图像的数字库,算是深度学习的 hello world


In [3]:
print(train_images.shape)
print(len(train_images))
train_labels

(60000, 28, 28)
60000


array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

数据集特征

- 训练集 60000 验证集 10000
- 图像是 28*28 的灰度图像,一个像素的灰度值在 0-255.
- 标签是 0-9 数字,每个图像对应一个标签


In [4]:
from tensorflow.keras import models
from tensorflow.keras import layers

model = models.Sequential([
    layers.Dense(512, activation='relu'),  #全连接层，512个节点
    layers.Dense(10, activation='softmax')  #全连接 输出层，10个节点，输出层全连接
])

2022-03-01 03:34:08.775178: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-03-01 03:34:08.781726: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-03-01 03:34:08.782230: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-03-01 03:34:08.783071: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate

神经网络的核心是层的堆叠.可以简单将神经网络理解为一层一层的筛子,一层一层的过滤数据,数据的可用性不断增加.

示例中包含了两个 Dense 全连接层.第二层作为输出,返回 10 个概率值,表面图片可能是 [0,9] 哪一个.


In [5]:
model.compile(  #编译模型
    optimizer='rmsprop',  #优化器
    loss='sparse_categorical_crossentropy',  #损失函数
    metrics=['accuracy'])  #指标

真正训练神经网络前,还需要 3 个参数

- 优化器: 模型根据数据选择优化的方向
- 损失函数: 基于数据计算当前网络的性能
- 指标: 训练过程中需要关注那些指标,这里只关心精度,分类正确占有的比例.


In [6]:
train_images = train_images.reshape(
    (60000, 28 * 28))  #把训练集的数据变成一个60000*784的数组)
train_images = train_images.astype('float32') / 255  #把数组元素转化为float32类型，值范围为0~1
test_images = test_images.reshape((10000, 28 * 28))  #把测试集的数据变成一个10000*784的数组)
test_images = test_images.astype('float32') / 255  #把数组元素转化为float32类型，值范围为0~1

在正式训练前,我们需要对数据进行预处理.

- [0,255] 的灰度值转换到 [0,1] 上
- 将所有数据都放到同一个矩阵


In [7]:
model.fit(train_images, train_labels, \
            epochs=5,batch_size=128)  #训练模型，训练5轮，每批128个样本

2022-03-01 03:35:03.282329: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/5
469/469 [==============================] - 4s 5ms/step - loss: 0.2600 - accuracy: 0.9245
Epoch 2/5
469/469 [==============================] - 2s 5ms/step - loss: 0.1047 - accuracy: 0.9685
Epoch 3/5
469/469 [==============================] - 3s 5ms/step - loss: 0.0682 - accuracy: 0.9799
Epoch 4/5
469/469 [==============================] - 2s 5ms/step - loss: 0.0496 - accuracy: 0.9849
Epoch 5/5
469/469 [==============================] - 2s 5ms/step - loss: 0.0377 - accuracy: 0.9884


现在可以开始 fit 拟合了,这里参数是训练 5 次,梯度下降的 batch_size 选择 128

训练时输出两个结果,在训练数据上 loss 损失 和 精度.这里第 5 轮.精度已经是 99.94%了


In [8]:
test_digits = test_images[:10]  #取前10个测试集图片
predictions = model.predict(test_digits)  #对测试集进行预测
predictions[0]  #预测结果

array([3.79033906e-08, 3.12442086e-11, 1.73598266e-06, 3.24615612e-05,
       1.32094780e-12, 2.60789790e-08, 3.01983085e-15, 9.99964714e-01,
       1.02589304e-07, 1.02932165e-06], dtype=float32)

取 10 个测试集样本,看看效果.


In [10]:
print(predictions[0].argmax())
predictions[0][7]

7


0.9999647

这里输出第一个的预测值和概率...直接是 1 了..😂

In [11]:
test_labels[0]

7

再看一下数据集,结果符合.~~(废话)~~

In [12]:
test_loss, test_acc = model.evaluate(test_images, test_labels)  #对测试集进行评估
test_acc  #测试集的准确率

313/313 [==============================] - 1s 4ms/step - loss: 0.0637 - accuracy: 0.9810


0.9810000061988831

最后验证一下在测试集上的准确度 98.22%.